<a href="https://colab.research.google.com/github/AlkaidCheng/GSoC2021_QMLHEP/blob/main/examples/QuplePQC_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.4.1
!pip install tensorflow-quantum
!pip install quple

     |████████████████████████████████| 394.3MB 41kB/s 
     |████████████████████████████████| 3.8MB 24.9MB/s 
     |████████████████████████████████| 471kB 52.6MB/s 
     |████████████████████████████████| 2.9MB 23.7MB/s 
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0
     |████████████████████████████████| 7.8MB 6.9MB/s 
     |████████████████████████████████| 1.3MB 39.8MB/s 
     |███████████████████

     |████████████████████████████████| 102kB 5.8MB/s 


In [ ]:
import sympy
import quple
import cirq
import tensorflow as tf

In [ ]:
# let's make some sympy expressions
feature_dimension = 4
x = sympy.symarray("x", feature_dimension)
formula = sympy.pi*(sympy.cos(x[0])*sympy.sin(x[1]))**2/(x[2]+x[3])
formula

pi*sin(x_1)**2*cos(x_0)**2/(x_2 + x_3)

In [ ]:
symbols = [i.name for i in x]
symbols

['x_0', 'x_1', 'x_2', 'x_3']

In [ ]:
from quple.interface.tfq.tf_resolvers import resolve_formula
# resolve the formula as tf operations
# here the order of the "symbols" will correspond to the index of the tf.Tensor which the symbol gets mapped to
# for example, if 
# symbols = ["a", "b", "c", "d"]
# and x = tf.Tensor([1, 2, 3, 4])
# then symbol "a" will map to the x[0] which equals 1 and "b" to x[1], and so on
resolved_formula = resolve_formula(formula, symbols)

In [ ]:
# try to get the value of the formula from a tensor variable
x = tf.Variable(tf.random.normal((feature_dimension,)))
x

<tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([-0.49065357, -0.3547651 , -0.6583589 , -0.26907787], dtype=float32)>

In [ ]:
resolved_formula(x)

<tf.Tensor: shape=(), dtype=float32, numpy=-0.31798917>

In [ ]:
# you can also use 2d tensor, but it will get resolved like
# pi*sin(x[1])*cos(x[0])/ (x[2]+x[3])
# so if your tensor shape is (batchsize, features), you will need to transpose it first, then transpose it back

# fortunately, there is another method to achieve that without much trouble:
# this method works on a list of formulas
from quple.interface.tfq.tf_resolvers import resolve_formulas
resolved_formulas = resolve_formulas([formula], symbols)

In [ ]:
batchsize = 10
x = tf.Variable(tf.random.normal((batchsize, feature_dimension)))
x

<tf.Variable 'Variable:0' shape=(10, 4) dtype=float32, numpy=
array([[ 0.34069398,  1.4260918 , -0.9071983 ,  0.28731903],
       [ 0.61207396,  0.3552132 ,  0.22679573,  0.9919552 ],
       [ 0.01560564, -0.37249205, -1.355871  ,  0.49101305],
       [ 0.3221842 ,  0.3426842 , -1.0314633 , -0.11221913],
       [ 0.42980465, -0.39307436, -0.3663515 , -1.0354388 ],
       [-0.3336734 , -0.38435593,  0.35889557,  0.17012145],
       [ 0.3865679 , -0.15573   ,  1.1938944 ,  0.8903954 ],
       [-0.76794577, -0.557606  , -0.07827922, -0.9453481 ],
       [-0.09783286,  0.23846461, -0.24306053, -0.7797711 ],
       [ 2.0285783 ,  0.9260963 , -1.5204161 ,  0.11615999]],
      dtype=float32)>

In [ ]:
resolved_formulas(x)

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[-4.4086027 ],
       [ 0.20886359],
       [-0.48100927],
       [-0.27905023],
       [-0.27170995],
       [ 0.74537843],
       [ 0.03110571],
       [-0.44467568],
       [-0.16973929],
       [-0.27916858]], dtype=float32)>

## Practical Example

In [ ]:
n_qubit = feature_dimension = 5

In [ ]:
from quple import ParameterisedCircuit
model_circuit = ParameterisedCircuit(n_qubit=n_qubit, copies=2, rotation_blocks=['RZ', 'RY'], entanglement_blocks=['XX'], entangle_strategy="alternate_linear")
model_circuit

(0, 0): ───Rz(θ_0)───Ry(θ_5)───XX──────────Rz(θ_14)────Ry(θ_19)──────────────XX──────────────────────
                               │                                             │
(0, 1): ───Rz(θ_1)───Ry(θ_6)───XX^(θ_10)───XX──────────Rz(θ_15)───Ry(θ_20)───XX^(θ_24)───XX──────────
                                           │                                             │
(0, 2): ───Rz(θ_2)───Ry(θ_7)───XX──────────XX^(θ_12)───Rz(θ_16)───Ry(θ_21)───XX──────────XX^(θ_26)───
                               │                                             │
(0, 3): ───Rz(θ_3)───Ry(θ_8)───XX^(θ_11)───XX──────────Rz(θ_17)───Ry(θ_22)───XX^(θ_25)───XX──────────
                                           │                                             │
(0, 4): ───Rz(θ_4)───Ry(θ_9)───────────────XX^(θ_13)───Rz(θ_18)───Ry(θ_23)───────────────XX^(θ_27)───

In [ ]:
from quple.data_encoding import EncodingCircuit
from quple import PauliBlock
data_circuit = EncodingCircuit(feature_dimension=feature_dimension, copies=2, rotation_blocks=['H', 'RZ'], entanglement_blocks=[PauliBlock('ZZ')], entangle_strategy="alternate_linear", 
                               encoding_map="cosine_product",
                               parameter_scale=1,      # by default will multiply every parameter by sympy.pi if parameter_scale is not specified
                               flatten_circuit=False)  # flattening the circuit will combine formulas into a single symbol so when you print the circuit it doesn't look quite good
data_circuit

(0, 0): ───H───Rz(x_0)───@───────────────────────────@───────────────────────────────────────────────────────────────────────────────H───Rz(x_0)───@─────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────
                         │                           │                                                                                             │                                 │
(0, 1): ───H───Rz(x_1)───X───Rz(cos(x_0)*cos(x_1))───X───────────────────────────@───────────────────────────@───────────────────────H───Rz(x_1)───X─────────Rz(cos(x_0)*cos(x_1))───X───────────────────────────@───────────────────────────@───────────────────────────
                                                                                 │                           │                                                                                                   │                           │
(0, 2): ───H───Rz(x_2)───@───────────────────────────────────────────────────@───X───Rz(cos(x_1)*cos(x_2))───X───────────────────────H───Rz(x_2)─────────────@───────────────────────────────────────────────@───X───Rz(cos(x_1)*cos(x_2))───X───────────────────────────
                         │                                                   │                                                                               │                                               │
(0, 3): ───H───Rz(x_3)───X───────────────────────────Rz(cos(x_2)*cos(x_3))───X───@───────────────────────────────────────────────────@───H─────────Rz(x_3)───X───────────────────────Rz(cos(x_2)*cos(x_3))───X───@───────────────────────────────────────────────────@───
                                                                                 │                                                   │                                                                           │                                                   │
(0, 4): ───H───Rz(x_4)───────────────────────────────────────────────────────────X───────────────────────────Rz(cos(x_3)*cos(x_4))───X───H─────────Rz(x_4)───────────────────────────────────────────────────────X───────────────────────────Rz(cos(x_3)*cos(x_4))───X───

In [ ]:
symbols = data_circuit.symbols
symbols

['x_0', 'x_1', 'x_2', 'x_3', 'x_4']

In [ ]:
data_circuit.flatten()
data_circuit

(0, 0): ───H───Rz(pi*<x_0/pi>)───@───────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────────────H───Rz(pi*<x_0/pi>)───@─────────────────────────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────────────────
                                 │                                   │                                                                                                                             │                                                 │
(0, 1): ───H───Rz(pi*<x_1/pi>)───X───Rz(pi*<cos(x_0)*cos(x_1)/pi>)───X───────────────────────────────────@───────────────────────────────────@───────────────────────────────H───Rz(pi*<x_1/pi>)───X─────────────────Rz(pi*<cos(x_0)*cos(x_1)/pi>)───X───────────────────────────────────@───────────────────────────────────@───────────────────────────────────
                                                                                                         │                                   │                                                                                                                                           │                                   │
(0, 2): ───H───Rz(pi*<x_2/pi>)───@───────────────────────────────────────────────────────────────────@───X───Rz(pi*<cos(x_1)*cos(x_2)/pi>)───X───────────────────────────────H───Rz(pi*<x_2/pi>)─────────────────────@───────────────────────────────────────────────────────────────@───X───Rz(pi*<cos(x_1)*cos(x_2)/pi>)───X───────────────────────────────────
                                 │                                                                   │                                                                                                               │                                                               │
(0, 3): ───H───Rz(pi*<x_3/pi>)───X───────────────────────────────────Rz(pi*<cos(x_2)*cos(x_3)/pi>)───X───@───────────────────────────────────────────────────────────────────@───H─────────────────Rz(pi*<x_3/pi>)───X───────────────────────────────Rz(pi*<cos(x_2)*cos(x_3)/pi>)───X───@───────────────────────────────────────────────────────────────────@───
                                                                                                         │                                                                   │                                                                                                           │                                                                   │
(0, 4): ───H───Rz(pi*<x_4/pi>)───────────────────────────────────────────────────────────────────────────X───────────────────────────────────Rz(pi*<cos(x_3)*cos(x_4)/pi>)───X───H─────────────────Rz(pi*<x_4/pi>)───────────────────────────────────────────────────────────────────────X───────────────────────────────────Rz(pi*<cos(x_3)*cos(x_4)/pi>)───X───

In [ ]:
# you can see that flatten() literally flatten the symbols
data_circuit.symbols

['<cos(x_0)*cos(x_1)/pi>',
 '<cos(x_1)*cos(x_2)/pi>',
 '<cos(x_2)*cos(x_3)/pi>',
 '<cos(x_3)*cos(x_4)/pi>',
 '<x_0/pi>',
 '<x_1/pi>',
 '<x_2/pi>',
 '<x_3/pi>',
 '<x_4/pi>']

In [ ]:
# by doing flatten(), an cirq.ExpressionMap is created which gives you a handle of the sympy formulas involved
data_circuit.expr_map

cirq.ExpressionMap({x_0/pi: <x_0/pi>, x_1/pi: <x_1/pi>, x_2/pi: <x_2/pi>, x_3/pi: <x_3/pi>, x_4/pi: <x_4/pi>, cos(x_0)*cos(x_1)/pi: <cos(x_0)*cos(x_1)/pi>, cos(x_2)*cos(x_3)/pi: <cos(x_2)*cos(x_3)/pi>, cos(x_1)*cos(x_2)/pi: <cos(x_1)*cos(x_2)/pi>, cos(x_3)*cos(x_4)/pi: <cos(x_3)*cos(x_4)/pi>})

In [ ]:
formulas = list(data_circuit.expr_map)
formulas

[x_0/pi,
 x_1/pi,
 x_2/pi,
 x_3/pi,
 x_4/pi,
 cos(x_0)*cos(x_1)/pi,
 cos(x_2)*cos(x_3)/pi,
 cos(x_1)*cos(x_2)/pi,
 cos(x_3)*cos(x_4)/pi]

In [ ]:
# you can then resolve the formulas in terms of tf operations using
resolved_formulas = resolve_formulas(formulas, symbols)

In [ ]:
# create your favourite training input
batchsize = 10
x = tf.random.normal((batchsize, feature_dimension))
x

<tf.Tensor: shape=(10, 5), dtype=float32, numpy=
array([[-0.24596362, -1.1355942 , -1.4965937 ,  1.6728464 , -2.0675292 ],
       [ 0.5458359 ,  0.76345634,  1.5090107 , -0.27239504,  0.789961  ],
       [-1.2302603 , -0.42953163,  1.6840651 ,  0.46873206, -0.19669703],
       [-0.02437594,  2.3881183 , -2.14611   , -0.23488878,  0.0228683 ],
       [-0.0609254 , -1.1647993 ,  2.0511315 ,  0.221681  ,  0.08062973],
       [-0.8195732 , -0.00395163, -1.6016334 ,  2.311174  ,  0.17437115],
       [ 0.3900983 , -1.5200758 , -0.27337474,  0.59688985,  0.37387502],
       [-1.1892382 , -1.0763328 , -0.4708972 , -0.37735718, -1.8763796 ],
       [ 1.0227927 ,  2.1483288 , -0.3803175 ,  0.36922193, -2.918509  ],
       [-1.0051631 ,  0.16521662,  1.5798893 ,  2.2749743 ,  0.75382626]],
      dtype=float32)>

In [ ]:
# here
# y[0][0] is the value of the 1st formula, i.e. x_0/pi for the first data batch
# y[0][6] is the value of the 6th formula, i.e. cos(x_0)*cos(x_1)/pi for the first data batch
# y[1][0] is the value of the 1st formula for the second data batch
y = resolved_formulas(x)
y

<tf.Tensor: shape=(10, 9), dtype=float32, numpy=
array([[-0.07829265, -0.36147085, -0.47638056,  0.5324835 , -0.65811497,
         0.13015851, -0.00240398,  0.00994866,  0.01545337],
       [ 0.17374496,  0.24301569,  0.480333  , -0.08670603,  0.2514524 ,
         0.19654801,  0.0189298 ,  0.01419938,  0.21578886],
       [-0.39160398, -0.13672416,  0.53605455,  0.14920205, -0.0626106 ,
         0.09665571, -0.03209705, -0.03270936,  0.27850172],
       [-0.0077591 ,  0.76016164, -0.683128  , -0.07476742,  0.0072792 ,
        -0.23207958, -0.16843599,  0.12631157,  0.30948824],
       [-0.01939316, -0.37076712,  0.6528954 ,  0.07056325,  0.02566524,
         0.12547845, -0.14348425, -0.0580884 ,  0.30951178],
       [-0.26087824, -0.00125784, -0.50981575,  0.7356695 ,  0.05550406,
         0.21725534,  0.00662033, -0.00981412, -0.21146517],
       [ 0.12417214, -0.48385513, -0.08701788,  0.18999593,  0.11900811,
         0.01492551,  0.25349373,  0.01553865,  0.24508324],
       [-0.37

# The PQC class
- It accepts the `model_circuit` and `data_circuit` as input
- `model_circuit` contains the weights of your pqc layer so this circuit should not contain composite formulas in its gate operations
- `data_circuit` is the circuit that transforms the classical input data into the input quantum state of the `model_circuit`, it can contain composite formmulas in its gate operations which correspond to the so-called "feature maps"

What it does:
- It flattens the `data_circuit` so that all formulas inside the circuit turns into symbols. This is important because tfq simulator (qsim) does not support resolving sympy formulas.
- It compiles the formulas inside the `data_circuit` in terms of a lambda function of composite tensorflow operations when the class is initialzed. 
- When the class is called, it transforms the `inputs` tensor into a tensor containing the resolved values of the formulas of the `data_circuit`. The tensor is then clipped to the weight tensor and passed as `symbol_values` to the tfq simulator.

In [ ]:
from quple.interface.tfq.layers.pqc import PQC

In [ ]:
qubits = quple.get_circuit_qubits(data_circuit)
readout = [cirq.Z(qubit) for qubit in qubits]

In [ ]:
pqc_layer = PQC(model_circuit, data_circuit, readout)

In [ ]:
x = tf.random.normal((batchsize, feature_dimension))
pqc_layer(x)

<tf.Tensor: shape=(10, 5), dtype=float32, numpy=
array([[ 0.27678746,  0.5200558 ,  0.01959522, -0.14102076,  0.17118864],
       [ 0.00913363,  0.6656975 ,  0.14828914, -0.27971178, -0.20858736],
       [-0.39217895,  0.17759456, -0.37867796, -0.27535462,  0.02611235],
       [-0.02902753,  0.6732651 ,  0.21796784, -0.33425954,  0.0056381 ],
       [-0.13300319,  0.20217375, -0.11104707,  0.26674023,  0.47732952],
       [-0.48761177,  0.25493357, -0.21529172, -0.0192354 , -0.5494061 ],
       [-0.39816177,  0.4321662 , -0.02633118, -0.02426801, -0.4981192 ],
       [ 0.09813245,  0.25100654, -0.3940597 , -0.10004365, -0.46641397],
       [-0.06610939,  0.00961988, -0.45314544, -0.00989364,  0.34783724],
       [-0.2253911 , -0.34409267, -0.45005468,  0.23739319, -0.4435135 ]],
      dtype=float32)>

In [ ]:
model = tf.keras.Sequential()
input_layer = tf.keras.layers.Input(shape=(feature_dimension), dtype=tf.float32)
model.add(input_layer)
model.add(pqc_layer)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
pqc (PQC)                    (None, 5)                 28        
Total params: 28
Trainable params: 28
Non-trainable params: 0
_________________________________________________________________


In [ ]:
output = model(x)
output

<tf.Tensor: shape=(10, 5), dtype=float32, numpy=
array([[ 0.27678746,  0.5200558 ,  0.01959522, -0.14102076,  0.17118864],
       [ 0.00913363,  0.6656975 ,  0.14828914, -0.27971178, -0.20858736],
       [-0.39217895,  0.17759456, -0.37867796, -0.27535462,  0.02611235],
       [-0.02902753,  0.6732651 ,  0.21796784, -0.33425954,  0.0056381 ],
       [-0.13300319,  0.20217375, -0.11104707,  0.26674023,  0.47732952],
       [-0.48761177,  0.25493357, -0.21529172, -0.0192354 , -0.5494061 ],
       [-0.39816177,  0.4321662 , -0.02633118, -0.02426801, -0.4981192 ],
       [ 0.09813245,  0.25100654, -0.3940597 , -0.10004365, -0.46641397],
       [-0.06610939,  0.00961988, -0.45314544, -0.00989364,  0.34783724],
       [-0.2253911 , -0.34409267, -0.45005468,  0.23739319, -0.4435135 ]],
      dtype=float32)>

# Validation
- We will validate the result by manually resolving the both `data_circuit` and `model_circuit` then measure the Z component of the spin of each qubit

In [ ]:
resolved_data_circuits = data_circuit.resolve_parameters(x.numpy())

In [ ]:
resolved_data_circuits[0]

(0, 0): ───H───Rz(-0.03π)────@────────────────@────────────────────────────────────────────H───Rz(-0.03π)───@──────────────────────────@────────────────────────────────────────────────
                             │                │                                                             │                          │
(0, 1): ───H───Rz(0.095π)────X───Rz(0.303π)───X────────────────@──────────────@────────────H───Rz(0.095π)───X─────────────Rz(0.303π)───X────────────────@──────────────@────────────────
                                                               │              │                                                                         │              │
(0, 2): ───H───Rz(0.054π)────@─────────────────────────────@───X───Rz(0.3π)───X────────────H───Rz(0.054π)─────────────────@─────────────────────────@───X───Rz(0.3π)───X────────────────
                             │                             │                                                              │                         │
(0, 3): ───H───Rz(-0.381π)───X────────────────Rz(0.115π)───X───@───────────────────────────@───H────────────Rz(-0.381π)───X────────────Rz(0.115π)───X───@───────────────────────────@───
                                                               │                           │                                                            │                           │
(0, 4): ───H───Rz(-0.33π)──────────────────────────────────────X──────────────Rz(0.059π)───X───H────────────Rz(-0.33π)──────────────────────────────────X──────────────Rz(0.059π)───X───

In [ ]:
resolved_model_circuit = model_circuit.resolve_parameters(pqc_layer.weights[0].numpy())
resolved_model_circuit

(0, 0): ───Rz(1.08π)────Ry(1.75π)────XX─────────Rz(1.33π)───Ry(1.68π)─────────────────XX──────────────────────
                                     │                                                │
(0, 1): ───Rz(0.217π)───Ry(0.877π)───XX^0.989───XX──────────Rz(0.962π)───Ry(1.64π)────XX^0.821────XX──────────
                                                │                                                 │
(0, 2): ───Rz(1.63π)────Ry(1.14π)────XX─────────XX^0.841────Rz(0.8π)─────Ry(0.57π)────XX──────────XX^-0.393───
                                     │                                                │
(0, 3): ───Rz(1.19π)────Ry(1.63π)────XX^-0.95───XX──────────Rz(1.69π)────Ry(0.823π)───XX^-0.079───XX──────────
                                                │                                                 │
(0, 4): ───Rz(1.91π)────Ry(1.5π)────────────────XX^-0.453───Rz(1.08π)────Ry(1.25π)────────────────XX^0.838────

In [ ]:
combined_circuits = [ParameterisedCircuit.from_cirq(resolved_data_circuit + resolved_model_circuit) for resolved_data_circuit in resolved_data_circuits]

In [ ]:
combined_circuits[0]

(0, 0): ───H───Rz(-0.03π)────@────────────────@────────────────────────────────────────────H───Rz(-0.03π)───@──────────────────────────@────────────────────────────────────────────────Rz(1.08π)────Ry(1.75π)────XX─────────Rz(1.33π)───Ry(1.68π)─────────────────XX──────────────────────
                             │                │                                                             │                          │                                                                          │                                                │
(0, 1): ───H───Rz(0.095π)────X───Rz(0.303π)───X────────────────@──────────────@────────────H───Rz(0.095π)───X─────────────Rz(0.303π)───X────────────────@──────────────@────────────────Rz(0.217π)───Ry(0.877π)───XX^0.989───XX──────────Rz(0.962π)───Ry(1.64π)────XX^0.821────XX──────────
                                                               │              │                                                                         │              │                                                     │                                                 │
(0, 2): ───H───Rz(0.054π)────@─────────────────────────────@───X───Rz(0.3π)───X────────────H───Rz(0.054π)─────────────────@─────────────────────────@───X───Rz(0.3π)───X────────────────Rz(1.63π)────Ry(1.14π)────XX─────────XX^0.841────Rz(0.8π)─────Ry(0.57π)────XX──────────XX^-0.393───
                             │                             │                                                              │                         │                                                             │                                                │
(0, 3): ───H───Rz(-0.381π)───X────────────────Rz(0.115π)───X───@───────────────────────────@───H────────────Rz(-0.381π)───X────────────Rz(0.115π)───X───@───────────────────────────@───Rz(1.19π)────Ry(1.63π)────XX^-0.95───XX──────────Rz(1.69π)────Ry(0.823π)───XX^-0.079───XX──────────
                                                               │                           │                                                            │                           │                                        │                                                 │
(0, 4): ───H───Rz(-0.33π)──────────────────────────────────────X──────────────Rz(0.059π)───X───H────────────Rz(-0.33π)──────────────────────────────────X──────────────Rz(0.059π)───X───Rz(1.91π)────Ry(1.5π)────────────────XX^-0.453───Rz(1.08π)────Ry(1.25π)────────────────XX^0.838────

In [ ]:
# the difference is due to the fact that "get_expectation" uses sampling whereas PQC by default use analytic method
print("result from validation: ")
print(combined_circuits[0].get_expectation(operators=readout, repetitions=1000000))
print()
print("result from PQC: ")
print(output[0])

result from validation: 
[[ 0.275906  0.520178  0.019066 -0.141302  0.170986]]

result from PQC: 
tf.Tensor([ 0.27678746  0.5200558   0.01959522 -0.14102076  0.17118864], shape=(5,), dtype=float32)


In [ ]:
print("result from validation: ")
print(combined_circuits[1].get_expectation(operators=readout, repetitions=1000000))
print()
print("result from PQC: ")
print(output[1])

result from validation: 
[[ 0.009552  0.665274  0.14919  -0.28005  -0.20802 ]]

result from PQC: 
tf.Tensor([ 0.00913363  0.6656975   0.14828914 -0.27971178 -0.20858736], shape=(5,), dtype=float32)


In [ ]:
print("result from validation: ")
print(combined_circuits[2].get_expectation(operators=readout, repetitions=1000000))
print()
print("result from PQC: ")
print(output[2])

result from validation: 
[[-0.391368  0.175924 -0.378638 -0.274302  0.02673 ]]

result from PQC: 
tf.Tensor([-0.39217895  0.17759456 -0.37867796 -0.27535462  0.02611235], shape=(5,), dtype=float32)


## Another way to validate

In [ ]:
unresolved_combined_circuit = ParameterisedCircuit.from_cirq(data_circuit_ + model_circuit)
unresolved_combined_circuit.flatten()
unresolved_combined_circuit

(0, 0): ───H───Rz(pi*<0.318309886183791*x_0>)───@──────────────────────────────────────────────────@────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────H───Rz(pi*<0.318309886183791*x_0>)───@───────────────────────────────────────────────────────────────────────────────@────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────Rz(pi*<θ_0/pi>)───Ry(pi*<θ_5/pi>)───XX──────────Rz(pi*<θ_14/pi>)───Ry(pi*<θ_19/pi>)──────────────────────XX──────────────────────
                                                │                                                  │                                                                                                                                                                                         │                                                                               │                                                                                                                                                                                            │                                                                    │
(0, 1): ───H───Rz(pi*<0.318309886183791*x_1>)───X───Rz(pi*<0.318309886183791*cos(x_0)*cos(x_1)>)───X──────────────────────────────────────────────────@──────────────────────────────────────────────────@──────────────────────────────────────────────H───Rz(pi*<0.318309886183791*x_1>)───X────────────────────────────────Rz(pi*<0.318309886183791*cos(x_0)*cos(x_1)>)───X──────────────────────────────────────────────────@──────────────────────────────────────────────────@──────────────────────────────────────────────────Rz(pi*<θ_1/pi>)───Ry(pi*<θ_6/pi>)───XX^(θ_10)───XX─────────────────Rz(pi*<θ_15/pi>)───Ry(pi*<θ_20/pi>)───XX^(θ_24)───XX──────────
                                                                                                                                                      │                                                  │                                                                                                                                                                                                                      │                                                  │                                                                                                  │                                                                    │
(0, 2): ───H───Rz(pi*<0.318309886183791*x_2>)───@─────────────────────────────────────────────────────────────────────────────────────────────────@───X───Rz(pi*<0.318309886183791*cos(x_1)*cos(x_2)>)───X──────────────────────────────────────────────H───Rz(pi*<0.318309886183791*x_2>)────────────────────────────────────@─────────────────────────────────────────────────────────────────────────────────────────────@───X───Rz(pi*<0.318309886183791*cos(x_1)*cos(x_2)>)───X──────────────────────────────────────────────────Rz(pi*<θ_2/pi>)───Ry(pi*<θ_7/pi>)───XX──────────XX^(θ_12)──────────Rz(pi*<θ_16/pi>)───Ry(pi*<θ_21/pi>)───XX──────────XX^(θ_26)───
                                                │                                                                                                 │                                                                                                                                                                           │                                                                                             │                                                                                                                                             │                                                                    │
(0, 3): ───H───Rz(pi*<0.318309886183791*x_3>)───X──────────────────────────────────────────────────Rz(pi*<0.318309886183791*cos(x_2)*cos(x_3)>)───X───@──────────────────

In [ ]:
symbols_names = unresolved_combined_circuit.raw_symbols
symbols_names

['x_0',
 'x_1',
 'x_2',
 'x_3',
 'x_4',
 'θ_0',
 'θ_1',
 'θ_2',
 'θ_3',
 'θ_4',
 'θ_5',
 'θ_6',
 'θ_7',
 'θ_8',
 'θ_9',
 'θ_10',
 'θ_11',
 'θ_12',
 'θ_13',
 'θ_14',
 'θ_15',
 'θ_16',
 'θ_17',
 'θ_18',
 'θ_19',
 'θ_20',
 'θ_21',
 'θ_22',
 'θ_23',
 'θ_24',
 'θ_25',
 'θ_26',
 'θ_27']

In [ ]:
tiled_up_weights = tf.tile([pqc_layer.weights[0]], [x.shape[0], 1])
tiled_up_parameters = tf.concat([x, tiled_up_weights], 1)
tiled_up_parameters

<tf.Tensor: shape=(10, 33), dtype=float32, numpy=
array([[-9.3632050e-02,  2.9914796e-01,  1.6890572e-01, -1.1966337e+00,
        -1.0382104e+00,  3.3875728e+00,  6.8188596e-01,  5.1163220e+00,
         3.7489159e+00,  5.9924626e+00,  5.4934864e+00,  2.7548647e+00,
         3.5951791e+00,  5.1244545e+00,  4.7064180e+00,  4.9888592e+00,
         5.0504436e+00,  4.8406892e+00,  1.5465403e+00,  4.1891580e+00,
         3.0228755e+00,  2.5119655e+00,  5.2999606e+00,  3.3889329e+00,
         5.2732410e+00,  5.1352916e+00,  1.7893504e+00,  2.5858903e+00,
         3.9274418e+00,  8.2111120e-01,  3.9211733e+00,  5.6067863e+00,
         8.3763891e-01],
       [ 5.2182311e-01,  1.9274540e-01, -8.6801529e-01, -1.7262475e-01,
        -4.6653348e-01,  3.3875728e+00,  6.8188596e-01,  5.1163220e+00,
         3.7489159e+00,  5.9924626e+00,  5.4934864e+00,  2.7548647e+00,
         3.5951791e+00,  5.1244545e+00,  4.7064180e+00,  4.9888592e+00,
         5.0504436e+00,  4.8406892e+00,  1.5465403e+00,  4.18

In [ ]:
_combined_circuits = unresolved_combined_circuit.resolve_parameters(tiled_up_parameters.numpy())

In [ ]:
_combined_circuits[0]

(0, 0): ───H───Rz(-0.03π)────@────────────────@────────────────────────────────────────────H───Rz(-0.03π)───@──────────────────────────@────────────────────────────────────────────────Rz(1.08π)────Ry(1.75π)────XX─────────Rz(1.33π)───Ry(1.68π)─────────────────XX──────────────────────
                             │                │                                                             │                          │                                                                          │                                                │
(0, 1): ───H───Rz(0.095π)────X───Rz(0.303π)───X────────────────@──────────────@────────────H───Rz(0.095π)───X─────────────Rz(0.303π)───X────────────────@──────────────@────────────────Rz(0.217π)───Ry(0.877π)───XX^0.989───XX──────────Rz(0.962π)───Ry(1.64π)────XX^0.821────XX──────────
                                                               │              │                                                                         │              │                                                     │                                                 │
(0, 2): ───H───Rz(0.054π)────@─────────────────────────────@───X───Rz(0.3π)───X────────────H───Rz(0.054π)─────────────────@─────────────────────────@───X───Rz(0.3π)───X────────────────Rz(1.63π)────Ry(1.14π)────XX─────────XX^0.841────Rz(0.8π)─────Ry(0.57π)────XX──────────XX^-0.393───
                             │                             │                                                              │                         │                                                             │                                                │
(0, 3): ───H───Rz(-0.381π)───X────────────────Rz(0.115π)───X───@───────────────────────────@───H────────────Rz(-0.381π)───X────────────Rz(0.115π)───X───@───────────────────────────@───Rz(1.19π)────Ry(1.63π)────XX^-0.95───XX──────────Rz(1.69π)────Ry(0.823π)───XX^-0.079───XX──────────
                                                               │                           │                                                            │                           │                                        │                                                 │
(0, 4): ───H───Rz(-0.33π)──────────────────────────────────────X──────────────Rz(0.059π)───X───H────────────Rz(-0.33π)──────────────────────────────────X──────────────Rz(0.059π)───X───Rz(1.91π)────Ry(1.5π)────────────────XX^-0.453───Rz(1.08π)────Ry(1.25π)────────────────XX^0.838────

In [ ]:
import tensorflow_quantum as tfq
expactation_layer = tfq.layers.SampledExpectation(backend=None)
results = expactation_layer(_combined_circuits, symbol_names=symbols_names, symbol_values=tiled_up_parameters, operators=readout, repetitions=1000000)
expectation = [result.numpy() for result in results]
expectation

[array([ 0.277612,  0.522318,  0.019718, -0.141122,  0.172258],
       dtype=float32),
 array([ 0.00938 ,  0.666714,  0.148524, -0.281518, -0.20737 ],
       dtype=float32),
 array([-0.392812,  0.176424, -0.37901 , -0.274292,  0.02502 ],
       dtype=float32),
 array([-0.025704,  0.671768,  0.21585 , -0.332944,  0.006172],
       dtype=float32),
 array([-0.133938,  0.203236, -0.11126 ,  0.268108,  0.47639 ],
       dtype=float32),
 array([-0.487624,  0.253796, -0.215434, -0.020086, -0.548782],
       dtype=float32),
 array([-0.39909 ,  0.431772, -0.02836 , -0.023788, -0.498424],
       dtype=float32),
 array([ 0.09802 ,  0.25085 , -0.394758, -0.09911 , -0.467404],
       dtype=float32),
 array([-0.06588 ,  0.00899 , -0.4552  , -0.010148,  0.349658],
       dtype=float32),
 array([-0.226212, -0.341538, -0.450216,  0.239512, -0.443626],
       dtype=float32)]

In [ ]:
# compared to 
output

<tf.Tensor: shape=(10, 5), dtype=float32, numpy=
array([[ 0.27678746,  0.5200558 ,  0.01959522, -0.14102076,  0.17118864],
       [ 0.00913363,  0.6656975 ,  0.14828914, -0.27971178, -0.20858736],
       [-0.39217895,  0.17759456, -0.37867796, -0.27535462,  0.02611235],
       [-0.02902753,  0.6732651 ,  0.21796784, -0.33425954,  0.0056381 ],
       [-0.13300319,  0.20217375, -0.11104707,  0.26674023,  0.47732952],
       [-0.48761177,  0.25493357, -0.21529172, -0.0192354 , -0.5494061 ],
       [-0.39816177,  0.4321662 , -0.02633118, -0.02426801, -0.4981192 ],
       [ 0.09813245,  0.25100654, -0.3940597 , -0.10004365, -0.46641397],
       [-0.06610939,  0.00961988, -0.45314544, -0.00989364,  0.34783724],
       [-0.2253911 , -0.34409267, -0.45005468,  0.23739319, -0.4435135 ]],
      dtype=float32)>